# Illustration of quantum operations on superposition of states

The quantum circuit presented in this notebook adds two 2-bit numbers into a 3-bit result.

This notebook illustrates the circuit where inputs presented is in superposition. With the input numbers in superposition, the result for all those input values is computed with a single run.

In [ ]:
import qckt
from qckt.backend import Qeng, Qdeb

Various qubit registers.

In [ ]:
nq = 9

s = [0, 1, 2]  # sum bits - Result of addition
c = [3, 4]     # carry bits, scratch pad register - Junk bits
b = [5, 6]     # input number B
a = [7, 8]     # input number A

Circuit to perform the addition operation

In [ ]:
addckt = qckt.QCkt(nq, nq)

# Create a 'full' superposition of input qubits
addckt.H(a)
addckt.H(b)

# run the addition 
for i in range(2):
    # print("Processing bit", i)
    addckt.CX(a[i],s[i])
    addckt.CX(b[i],s[i])
    if i != 0:
        addckt.CX(c[i-1],s[i])
    addckt.CCX(a[i],b[i],c[i])
    if i != 0:
        addckt.X(s[i])
        addckt.CCX(s[i],c[i-1],c[i])
        addckt.X(s[i])
addckt.CX(c[1],s[2])

# Clean up the junk bits
for i in reversed(range(2)):
    if i != 0:
        addckt.X(s[i])
        addckt.CCX(s[i],c[i-1],c[i])
        addckt.X(s[i])
    addckt.CCX(a[i],b[i],c[i])
# addckt.M([8,7,6,5,2,1,0])
# addckt.M([5,6,7,8],[5,6,7,8])

addckt.draw()

Run the circuit

In [ ]:
job = qckt.Job(addckt, qtrace=True)
bk = Qdeb()  # honors qtrace, allows reading state vector
# bk = Qeng()  # ignores qtrace, does not allow reading state vector
bk.runjob(job)

Readout the final state, it sows all additions performed in 'parallel'.

In [ ]:
print("READ OUT STATE VECTOR: ")
print(job.get_svec())

Print out the execution statistics.

In [ ]:
print('READ OUT RUNSTATS')
job.print_runstats()